# Processor

In [ ]:
import torch
from torchvision.models import MobileNetV2
import time

num_class = 4 # 0, 1, 2, 3
weight_path = ''
image_size = (224, 224)

In [ ]:
model = MobileNetV2(num_class)
t1 = time.time()
model.load_state_dict(torch.load(weight_path, map_location='cpu')) # Load weight
t2 = time.time()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Chọn device nếu là GPU thì sẽ chuyển sang GPU
model.to(device)
model.eval()
print(f'Load weight: {t2 - t1}s')

## 1. Preprocess

In [ ]:
def _preprocess(card_infos):
    images = [card_info.image for card_info in card_infos]
    samples = [cv2.resize(image, image_size) for image in images]
    samples = np.array(samples)
    samples = torch.from_numpy(samples).to(device).to(torch.float)
    samples = samples.permute(0, 3, 1, 2)
    samples = (samples - samples.mean()) / samples.std()
    return card_infos, samples

## 2. Process

In [ ]:
def _process(card_infos, samples):
    with torch.no_grad():
        preds = model(samples).softmax(dim=1)
    return card_infos, preds

## 3. Postprocess


In [ ]:
def _postprocess(card_infos, preds):
    #preds: probabilities of each class
    images = [card_info.image for card_info in card_infos]
    # Rotate image 90, 180, 270 degree
    rotated_images = [np.rot90(image, k = -pred.argmax().item()) for (pred, image) in zip(preds, images)]
    # argmax: get idx at highest value
    rotated_angles = [pred.argmax().item() * 90 for pred in preds]
    scores = [pred[pred.argmax()].item() for pred in preds]
    return card_infos, rotated_images, rotated_angles, scores

# Stage

## 1. Preprocess

In [ ]:
def preprocess(card_infos):
    if __debug__:
        for i, card_info in enumerate(card_infos):
            assert type(card_info.image).__name__ == 'ndarray', f'Image #{i} must be an ndarray.'
            assert card_info.image.ndim == 3, f'Image #{i} must be a 3D ndarray.'
            assert card_info.image.shape[-1] == 3, f'Image #{i} must have 3 channels.'

    return card_infos,

## 2. Process

In [ ]:
import cv2
import numpy as np

def process(card_infos):
    card_infos, samples = _preprocess(card_infos)
    card_infos, preds = _process(card_infos, samples)
    card_infos, rotated_images, rotated_angles, scores = _postprocess(card_infos, preds)
    return card_infos, rotated_images, rotated_angles, scores

## 3. Postprocess

In [ ]:
def postprocess(card_infos, rotated_images, rotated_angles, scores):
    for card_info, rotated_image, rotated_angle, score in zip(card_infos, rotated_images, rotated_angles, scores):
        card_info.image = rotated_image
        card_info.angle = rotated_angle
    return card_infos,

# TEST

In [ ]:
from enum import Enum


class DAO:
    @staticmethod #Khong dung self
    def _asdict(obj):
        if isinstance(obj, dict):
            return {key: DAO._asdict(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [DAO._asdict(element) for element in obj]
        elif isinstance(obj, tuple):
            return tuple([DAO._asdict(element) for element in obj])
        elif isinstance(obj, (int, float, str, bool)):
            return obj
        elif isinstance(obj, Enum):
            return obj.value
        elif isinstance(obj, DAO):
            return obj.asdict()
        else:
            raise ValueError('Unsupported type {}.'.format(type(obj)))

    def asdict(self):
        return {key: DAO._asdict(value) for key, value in self.__dict__.items() if value is not None}

    def __repr__(self):
        _repr = f'{self.__class__.__name__}('
        for key, value in self.__dict__.items():
            _repr += f'{key}={value}, '
        _repr = f'{_repr[:-2]})'

        return _repr

In [ ]:
class CardInfo(DAO):
    def __init__(self, 
                 image=None, 
                 angle: int=None):
        self.image = image
        self.angle = angle

In [ ]:
image = cv2.imread('test_images/input/warped_card.jpg')
card_info = CardInfo()

card_infos = []
card_info.image = image
card_infos.append(card_info)

In [ ]:
cv2.imshow('original image', image)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
card_infos, = preprocess(card_infos)
card_infos, rotated_images, rotated_angles, scores = process(card_infos)
card_infos, = postprocess(card_infos, rotated_images, rotated_angles, scores)

In [ ]:
for card_info in card_infos:
    cv2.imshow('rotated image', card_info.image)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [ ]:
card_infos